## 3_Convert_ONNX_to_TensorRT

This notebook will convert ONNX model to TensorRT serialized engine.

The batch size is choosen to be 1.
Conversion from ONNX to TensorRT would use the ONNXClassifierWrapper from Nvidia TensorRT repository.

https://github.com/NVIDIA/TensorRT/tree/master/quickstart/IntroNotebooks

Please see onnx_helper.py.
Important modification to note is, explicit batch must be set.

**explicit_batch = 1 << (int)(trt.NetworkDefinitionCreationFlag.EXPLICIT_BATCH)**

We can also use the trtexec command to convert.

trtexec --onnx=[ONNX file name.onnx] --saveEngine=[TensorRT engine.trt]  --explicitBatch

### Jetson Nano Specific Information:
 In Jetson Nano, during the conversion, RAM might run out.
 Hence please set up swap space in the device.
 
 Please refer to the setup instructions in this repository.

In [1]:
#Import packages
import numpy as np
from onnx_helper import ONNXClassifierWrapper,convert_onnx_to_engine
import torch
import json
from PIL import Image
from torchvision import transforms

In [2]:
#Set constants
BATCH_SIZE=1
N_CLASSES=1000
PRECISION=np.float32
image_size=224

In [3]:
#PyTorch transform
tfms=transforms.Compose([transforms.Resize(image_size),
                         transforms.CenterCrop(image_size),
                         transforms.ToTensor(),
                         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

In [7]:
img=Image.open('panda.jpg')
img=tfms(img)#.unsqueeze(0)

img=torch.unsqueeze(img, 0)
print("Img shape",img.shape)
BATCH_SIZE=1
dummy_batch=np.zeros((BATCH_SIZE,3,224,224))
for idx in range(BATCH_SIZE):
    dummy_batch[idx]=img
print(dummy_batch.shape)

Img shape torch.Size([1, 3, 224, 224])
(1, 3, 224, 224)


In [5]:
ONNX_PATH='efficientnetb2_batch1.onnx'
BATCH_SIZE=1
TRT_PATH='efficientnetb2_batch1.trt'
trt_engine=convert_onnx_to_engine(ONNX_PATH, TRT_PATH)

Parsing ONNX file.
Building TensorRT engine. This may take a few minutes.
